In [ ]:
import numpy as np # linear algebra
import pandas as pd 
import pickle
import os
import glob
import matplotlib.pyplot as plt
from datetime import datetime
import io
import math
import hw_functions
import features

model_list = ['MNIST_CNN*','AlexNet*','CIFAR10*','InceptionV3*',
                     'VGG19*','ResNet50*','InceptionResNetV2*','LeNet5*',
                   'ResNetSmall*']
Instances =  [ 'p100' ,'p3', 'g4dn', 'g5' ]
columns_name = ['Train_Name','Instances','model','batchsize','datasize','totaldata','total_latency_second','one_epoch_average_second'] + features.describe_columns + features.not_move_value_columns 

df = pd.DataFrame(columns = columns_name)  # 파일하나당 한줄씩 저장해줄것

In [ ]:
for Instance_name in Instances:  # 인스턴스별로
    print(Instance_name)
    Instance = '../DataSet/RawData/GPU/'+Instance_name
    
    # DATA.csv 전처리
    Data_T = pd.read_csv(Instance+'/Data.csv')
    Data_T ,Data_new_timestamp = hw_functions.Datacsv_preprocess(Data_T)
    Data_T= Data_T[Data_T.columns.difference(['hour', 'minute', 'second' , 'millisecond' , 'hms'])]
    Data_T_columns = [c for c in Data_T.columns if c.lower()[:7] != 'retired']      # retired_pages.double_bit.count/retired_pages.single_bit_ecc.count [N/A], 0 으로 인스턴스별로 다르게 출력됨
    Data_T_columns = [c for c in Data_T_columns if c.lower()[:3] != 'ecc']
    Data_T = Data_T[Data_T_columns]
    new_columns = []
    for c in Data_T.columns:
        c = c.replace(r'[', '')
        c = c.replace(r']', '')
        c = c.replace(r'.', '_')
        c = c.replace(r'%', '')
        new_columns.append(c)
    Data_T.columns=new_columns
    Data_T = Data_T.reset_index(drop = True)
    Data_T = Data_T.astype({'clocks_applications_graphicsMHz':'float'})
    Data_T = Data_T.astype({'clocks_applications_memoryMHz':'float'})
    Data_T = Data_T.astype({'clocks_current_graphicsMHz':'float'})
    Data_T = Data_T.astype({'clocks_current_memoryMHz':'float'})
    Data_T = Data_T.astype({'clocks_max_graphicsMHz':'float'})
    Data_T = Data_T.astype({'clocks_max_memoryMHz':'float'})
    Data_T = Data_T.astype({'clocks_max_smMHz':'float'})
    Data_T = Data_T.astype({'memory_freeMiB':'float'})
    Data_T = Data_T.astype({'memory_totalMiB':'float'})
    Data_T = Data_T.astype({'memory_usedMiB':'float'})
    Data_T = Data_T.astype({'power_max_limitW':'float'})
    Data_T = Data_T.astype({'power_min_limitW':'float'})

    # 움직이는 매트릭값들 판별
#     timeseries_columns = list(Data_T.columns)
#     for i in (Data_T.columns):
#         if( len(Data_T[i].value_counts()) == 1):
#             timeseries_columns.remove(i)

    for model in model_list: # 모델별로 실행
        try:  # 해당모델없으면 에러무시하고 넘어가기
            filenames = glob.glob(str(Instance)+'/'+str(model) )        
            for file in filenames: # 해당 모델이름으로 필터링된 파일 모두살펴보기
                print(file) #이 파일위주로 볼것이다.
    
                # 모델별로 epoch 시작과끝 기록한 파일열어서 array 에 저장
                epoch_start_timestamp_array, epoch_end_timestamp_array, epoch_start_end_timestamp_Dataframe = hw_functions.per_model_csv(file)
                print( '[총 에포크수]  : ' + str(  len(epoch_start_timestamp_array)))
                
                # epoch 시작 끝지점이 Data_T 랑 완전히 같지않기 때문에 가장 가까운 지점만 골라내줌 
                epoch_start_timestamp_nearest, epoch_end_timestamp_nearest, epoch_start_list, epoch_end_list = hw_functions.epoch_start_end_to_list(Data_T, epoch_start_timestamp_array ,epoch_end_timestamp_array)
                
                # Data_T2 : 해당 모델 데이터만 가지고 살펴본다 
                # 시작한지점이랑 끝난지점 기준으로 데이터 앞뒤 잘라내줌.
                Data_T2 = Data_T[epoch_start_list[0]:epoch_end_list[len(epoch_end_list)-1]]

                # 1th
                # Instances / model / batchsize / datasize / totaldata / total_latency_second / one_epoch_average_second
                Train_Name = Instance_name + '_' + file[file.rfind('/')+1:-4]
                Instances = Instance_name
                save_name = str(file[int(file.rfind('/')+1):-4]) 
                model =  str(save_name[:save_name.find('_batchsize')]) # MNIST_CNN예외처리
                save_name = save_name[save_name.find('_batchsize')+1:]
                batchsize =  str(save_name[save_name.find('batchsize')+9:save_name.find('_')])
                save_name = save_name[save_name.find('_')+9:]
                datasize =  str(save_name[:save_name.find('_')])
                save_name = save_name[save_name.find('_')+1:]
                totaldata =  str(save_name[save_name.rfind('_totaldata')+10:])

                with open(file, 'rb') as fr:
                    epoch_start_end_timestamp = pickle.load(fr)

                    start = datetime.strptime(epoch_start_end_timestamp[0][0],'%Y-%m-%d %H:%M:%S.%f')
                    end = datetime.strptime(epoch_start_end_timestamp[len(epoch_start_end_timestamp)-1][1],'%Y-%m-%d %H:%M:%S.%f')
                    total_latency_second = end.timestamp() - start.timestamp()
                    one_epoch_average_second = total_latency_second/10
                
                # 2th
                describe_columns_value = [] 
                describe_columns=[]
                column_name = []
                one_hot_encoding_onlytwo_columns = []
                one_hot_encoding_onlytwo_value = []

                Data_T2_desribe_result= Data_T2.describe() 
                # describe 에서 뺴줌
                del_describe = ['new_timestamp_gpu','accounting_buffer_size','clocks_max_graphicsMHz','clocks_max_memoryMHz','clocks_max_smMHz','pcie_link_gen_current','pcie_link_gen_max','pcie_link_width_current','pcie_link_width_max','count','index','inforom_ecc','inforom_oem','clocks_applications_graphicsMHz','clocks_applications_memoryMHz','clocks_current_memoryMHz','power_max_limitW','power_min_limitW','serial']                    
                for d in del_describe:
                    del Data_T2_desribe_result[d]  

                d= Data_T2_desribe_result[1:].to_dict(orient='dict')
                for key in d.keys():
                    for subkey in d[key].keys():
                        column_name = key +'_'+ subkey
                        globals()[column_name] = d[key][subkey]
                        describe_columns.append(column_name)
                        describe_columns_value.append(globals()[column_name])
                
                
                # 변화안하는 nvidia-smi 값도 넣어줌 'not_move_value_columns'
                not_move_value_columns_values = []
                for i in features.not_move_value_columns:
                    not_move_value_columns_values.append(Data_T[i].value_counts().index[0])

                # 위에껀 nvidia-smi 전부다
                #columns_name_value = [Instances,model,batchsize,datasize,totaldata,total_latency_second,one_epoch_average_second] + describe_columns_value  + not_move_value_columns_values + one_hot_encoding_onlytwo_value                  
                columns_name_value = [Train_Name,Instances,model,batchsize,datasize,totaldata,total_latency_second,one_epoch_average_second] + describe_columns_value  + not_move_value_columns_values                   
                df.loc[df.shape[0]] = columns_name_value
                print("END")
        except:
            pass

In [ ]:
devicequery_result = pd.read_csv('./devicequery_result.csv')
devicequery_result = devicequery_result.drop(index=11, axis=0)  # 숫자아닌값 제외해줌

for i in (list(devicequery_result['features'])):  # 열 붙이기
    df[i] = 1
for i in list(df['Instances'].value_counts().keys()):
    for j in (list(devicequery_result['features'])):
        print(j)
        df_new = float( devicequery_result [devicequery_result['features'] == j][i])     
        df.loc[df.Instances ==i,j]= df_new

In [ ]:
df.to_csv('./nvidiasmi_devicequery_all.csv',index = False)